In [ ]:
# Install the required libraries
pip install nltk

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
import nltk
from nltk.corpus import wordnet as wn
import random
import pandas as pd
import tqdm

Create the direct relations dataset. Here we will 

In [ ]:
import nltk
from nltk.corpus import wordnet as wn
import random
import pandas as pd
import tqdm


# Obter todas as synsets em WordNet
all_synsets = list(wn.all_synsets())

num_iterations = len(all_synsets)
# Criar uma lista para armazenar os dataframes
dataframes = []


# Iniciar a iteração
for iteration in tqdm.tqdm(range(num_iterations), desc='Progress'):

    # Obter uma synset aleatória
    synset = all_synsets[iteration]

    hypernyms = synset.hypernyms()
    hypernyms_data = {
        'Definição_Synset': [synset.definition() for _ in range(len(hypernyms))],
        'ID_Synset': [synset.name() for _ in range(len(hypernyms))],
        'Definição_Relacionada': [h.definition() for h in hypernyms],
        'ID_Relacionada': [h.name() for h in hypernyms],
        'Relacao': ['Hypernyms' for _ in range(len(hypernyms))]
    }
    hypernyms_df = pd.DataFrame(hypernyms_data)
    dataframes.append(hypernyms_df)

    #append de holonym data
    holonyms = synset.part_holonyms() + synset.substance_holonyms() + synset.member_holonyms()
    holonyms_data = {
        'Definição_Synset': [synset.definition() for _ in range(len(holonyms))],
        'ID_Synset': [synset.name() for _ in range(len(holonyms))],
        'Definição_Relacionada': [h.definition() for h in holonyms],
        'ID_Relacionada': [h.name() for h in holonyms],
        'Relacao': ['Holonyms' for _ in range(len(holonyms))]
    }
    holonyms_df = pd.DataFrame(holonyms_data)
    dataframes.append(holonyms_df)




Progress: 100%|██████████| 117659/117659 [00:47<00:00, 2501.97it/s]


In [ ]:
combined_df = pd.concat(dataframes)

# Remove any occasional duplicate records
df_no_duplicates = combined_df.drop_duplicates()

df_no_duplicates.to_csv('direct_relations.csv', index=False)

In [ ]:
class_counts = df_no_duplicates['Relacao'].value_counts()
class_counts

Hypernyms    88529
Holonyms     22146
Name: Relacao, dtype: int64

Indirect dataset

In [ ]:
all_synsets = list(wn.all_synsets())

In [ ]:
import numpy as np
import nltk
from nltk.corpus import wordnet as wn

def build_hypernym_matrix():
    synsets = list(wn.all_synsets())
    num_synsets = len(synsets)
    hypernym_matrix = np.zeros((num_synsets, num_synsets), dtype=int)

    def add_hypernyms(synset, hypernym_indices):
        for hypernym in synset.hypernyms():
            hypernym_index = synsets.index(hypernym)
            hypernym_indices.add(hypernym_index)
            add_hypernyms(hypernym, hypernym_indices)

    for i, synset in enumerate(synsets):
        print(f"Processing synset {i + 1} of {num_synsets}...")
        hypernym_indices = set()
        add_hypernyms(synset, hypernym_indices)
        for j in hypernym_indices:
            hypernym_matrix[i, j] = 1

    return hypernym_matrix

hypernym_matrix = build_hypernym_matrix()
print(hypernym_matrix)


Progress:  55%|█████▌    | 65073/117659 [29:53<2:24:09,  6.08it/s]